In [1]:
import os

In [2]:
%pwd

'c:\\Users\\timmy\\Downloads\\End to End ML projects\\End-to-End-Classification-with-Mlflow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\timmy\\Downloads\\End to End ML projects\\End-to-End-Classification-with-Mlflow'

In [5]:
import tensorflow as tf

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_url: str
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifier.constants import  *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
            self, 
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_cofig(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/chest-cancer-data/",
            mlflow_url="https://dagshub.com/timothyafolami/End-to-End-Classification-with-Mlflow.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE, 
            params_batch_size=self.params.BATCH_SIZE,
        )
        return eval_config

In [11]:
# compoonents

import tensorflow as tf
import pathlib as Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

from cnnClassifier.constants import  *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [12]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
    
    def _valid_generator(self):

        datagenerator_kwargs = dict(rescale=1./255, validation_split=.20)
        
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {
            "loss": self.score[0],
            "accuracy": self.score[1]
        }
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_url)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})

            # model registry does not work with file store
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [13]:
# pipeline

try:
    config_manager = ConfigurationManager()
    eval_config = config_manager.get_evaluation_cofig()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2024-01-15 05:05:57,883: INFO: cnnClassifierLogger: Yaml file loaded from config\config.yaml loaded successfully]
[2024-01-15 05:05:57,892: INFO: cnnClassifierLogger: Yaml file loaded from params.yaml loaded successfully]
[2024-01-15 05:05:57,894: INFO: cnnClassifierLogger: Created directory at artifacts]
Found 68 images belonging to 2 classes.
9/9 [==============================] - 9s 961ms/step - loss: 21.8131 - accuracy: 0.5735
[2024-01-15 05:06:07,849: INFO: cnnClassifierLogger: Saved json file at scores.json]


2024/01/15 05:06:09 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-01-15 05:06:11,537: WARNING: absl: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\timmy\AppData\Local\Temp\tmpixwv9zvf\model\data\model\assets
[2024-01-15 05:06:12,259: INFO: tensorflow: Assets written to: C:\Users\timmy\AppData\Local\Temp\tmpixwv9zvf\model\data\model\assets]


c:\Users\timmy\anaconda3\envs\cancer-pred\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/01/15 05:07:07 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 8
Created version '8' of model 'VGG16Model'.
